# AcuityMD Challenge

Below is some code that simply loads the data stored in `database.db`, so that you can execute some SQL queries using the pandas library. 

## 1. Find the Top HCPs

### 1.1 Install Libraries

Note, please do not use any additional libraries. Your solution should be completed in SQL.

In [ ]:
%pip install pandas
import pandas as pd
import sqlite3

### 1.2 Implementation

#### 1.2.1 Check the table names

In [ ]:

con = sqlite3.connect('database.db')
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

#### 1.2.2 Make SQL Query

In [ ]:
df = pd.read_sql_query("SELECT hcp, soc, volume FROM ProcedureData LIMIT 10", con)
print(df.head(10))

### 1.3 Notes

Please enter any assumptions that you have made in this part in this section.

## 2. Written Extension

In this step, we would like you to add the answer to the following questions in this section.
There can be multiple ways to prioritize HCPs to target, for example one might use an "upsell" strategy, which would prioritize HCPs with the most existing sales.
- How would you change your solution to handle returning the "top" HCPs based on multiple strategies, for example both "upsell" and "elephant"? Describe your approach, without writing any code. How would you consider sorting HCPS when using multiple strategies? 
Describe your approach, without writing any code. Given this description, an engineer should be able to implement a solution. Please keep your response under 500 words.